In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

<body>
train 인수 : True면 학습 데이터를 불러오고 False면 테스트 데이터를 불러온다.<br>
transform 인수 : 데이터에 대한 변형<br>
target_transform : 라벨에 대한 변형<br>
download : 현재 경로에 MINST 데이터가 없을 경우에 다운로드하겠다.<br>
</body>

In [5]:
mnist_train = dset.MNIST("./",train=True,transform = transforms.ToTensor(),target_transform=None,download=True)
mnist_test = dset.MNIST("./",train = False, transform = transforms.ToTensor(), target_transform = None,download=True)

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
#num_worker : 데이터를 묶을 때 사용할 프로세스의 갯수
#drop_last : 묶고 남는 데이터는 버릴지 여부

test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)




In [21]:
for j,[image,label] in enumerate(train_loader):
    print(label)
    if j == 0:
        break

tensor([9, 6, 2, 1, 8, 6, 3, 8, 5, 0, 0, 0, 1, 1, 1, 0, 3, 8, 8, 3, 9, 2, 0, 1,
        5, 3, 7, 6, 7, 5, 2, 8, 3, 8, 3, 4, 5, 4, 5, 5, 3, 1, 3, 1, 6, 3, 5, 6,
        9, 8, 1, 1, 6, 6, 6, 4, 3, 1, 3, 9, 3, 3, 5, 4, 9, 6, 4, 9, 5, 4, 4, 7,
        4, 3, 0, 8, 6, 4, 2, 0, 5, 0, 9, 3, 1, 1, 9, 4, 7, 2, 9, 2, 6, 1, 8, 9,
        2, 1, 4, 2, 6, 1, 7, 0, 5, 5, 8, 2, 8, 9, 8, 1, 1, 6, 9, 8, 4, 7, 7, 1,
        4, 9, 8, 6, 7, 0, 2, 8, 0, 4, 8, 0, 8, 2, 5, 3, 4, 9, 7, 0, 4, 1, 7, 1,
        9, 0, 5, 0, 5, 9, 5, 7, 7, 7, 6, 9, 7, 2, 1, 8, 4, 4, 3, 9, 0, 0, 0, 2,
        9, 1, 7, 7, 6, 7, 6, 2, 6, 6, 7, 4, 8, 9, 8, 4, 8, 0, 0, 1, 2, 1, 5, 3,
        0, 6, 0, 9, 5, 4, 8, 8, 8, 3, 3, 9, 7, 7, 2, 0, 5, 5, 6, 0, 7, 5, 8, 0,
        7, 0, 1, 2, 0, 5, 5, 8, 5, 0, 6, 5, 5, 8, 8, 5, 0, 4, 8, 9, 4, 9, 0, 8,
        4, 3, 1, 6, 1, 8, 5, 7, 6, 4, 2, 8, 7, 7, 2, 5])


<h2>CNN모델</h2>

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__() #CNN의 부모 클래스인 nn.Module을 초기화하는 역할
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),#5는 임의로 그냥 설정하였음.
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),#kernel_size: 풀링 연산 할 때 한번에 훑는 영역의 크기, #stride, #padding
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),#output size: [batch_size,64,6,6]
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)#output size : [batch_size,10]
        )
        
    def forward(self,x):#연산을 순차적으로 실행하여 결괏값만 리턴하도록 정의
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

<h2>모델 초기화와 손실함수 지정과 optimizer지정</h2>

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device) #모델을 올려줌
loss_func = nn.for j,[image,label] in enumerate(train_loader):() #classification에서는 crossentorpy loss function을 사용하는 게 좋음
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
loss_arr = []
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        